In [1]:
import os
import ocrmypdf
from google import genai
import chardet
from pathlib import Path
import time

In [6]:
# Replace API_KEY with your personal key
API_KEY = open("keyfile.txt", "r").read()

In [10]:
OCR_INPUT = "./ocr_input"      # Input folder for pdf files
OCR_OUTPUT = "./ocr_output"
LLM_INPUT = "./llm_input"   # Output folder for pdf and txt files
LLM_OUTPUT = "./llm_output"   # Output folder for pdf and txt files
LANGUAGE = "deu"             # OCR language: deu = German, eng = English, etc.

## Generate Input Data for LLM

In [ ]:
# Find all pdf files in input folder
for filename in os.listdir(OCR_INPUT):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(OCR_INPUT, filename)
        output_pdf_path = os.path.join(OCR_OUTPUT, filename)  # optional
        txt_filename = os.path.splitext(filename)[0] + ".txt"
        txt_path = os.path.join(OCR_OUTPUT, txt_filename)

        print(f"Processing: {filename}")
        
        try:
            ocrmypdf.ocr(
                pdf_path,
                output_pdf_path,
                sidecar=txt_path,
                language=LANGUAGE,
                force_ocr=True  # Falls PDF schon Text hat, trotzdem neu erkennen
            )
            print(f"-> Done: {txt_filename}")
        except Exception as e:
            print(f"Error at {filename}: {e}")

print("All Files Processed!")


In [ ]:
"""def convert_to_utf8(input_path, output_path):
    # Load binaries of file
    raw_bytes = Path(input_path).read_bytes()

    # Recognize encoding
    detection = chardet.detect(raw_bytes)
    encoding = detection["encoding"] or "utf-8"
    print(f"Detected encoding: {encoding}")

    # Decode (replace unsafe chars)
    decoded_text = raw_bytes.decode(encoding, errors="replace")

    # Repair Mojibake
    cleaned_text = fix_text(decoded_text)

    cleaned_text = re.findall("\w", cleaned_text, re.ASCII)

    # Save as UTF-8
    Path(output_path).write_text(cleaned_text, encoding="utf-8")
    print(f"Saved UTF-8 encoded file to: {output_path}")"""


In [11]:
data = []
for filename in os.listdir(LLM_INPUT):
	path = os.path.join(LLM_INPUT, filename)
	# Load binaries of file
	raw_bytes = Path(LLM_INPUT+"/"+str(filename)).read_bytes()
    # Recognize encoding
	encoding = chardet.detect(raw_bytes)["encoding"] or "utf-8"
	print(f"Detected encoding: {encoding}")
    # Decode (replace unsafe chars)
	decoded = raw_bytes.decode(encoding, errors="replace")
	print(filename+": "+str(len(decoded))+" characters")
	for i in range(int(len(decoded)/20000)+1):
		data.append(decoded[i*20000:(i+1)*20000])

Detected encoding: utf-8
Lehrvideos.txt: 45163 characters


## LLM Inference

In [12]:
# Set API Key
client = genai.Client(api_key=API_KEY)

# API Call
def generate_text(prompt_text:str) -> str:
    """Generates text response using the configured Gemini model."""
    response = client.models.generate_content(
        model="gemini-2.5-flash-preview-04-17",
        contents=prompt_text,
        config=genai.types.GenerateContentConfig(
            thinking_config=genai.types.ThinkingConfig(
            thinking_budget=4096
            )
        )
    )
    print("Prompt tokens:",response.usage_metadata.prompt_token_count,
          "| Thoughts tokens:",response.usage_metadata.thoughts_token_count,
          "| Output tokens:",response.usage_metadata.candidates_token_count,
          "| Total tokens:",response.usage_metadata.total_token_count)

    return response.text

#### Retrieve Information from Input Data

In [13]:
prompt = """Du erhälst die Inhalte einer Vorlseung an einer Universität als Eingabestring.
	Du sollst die Vorlesung zusammenfassen, indem du ausschließlich akademisch und thematisch wichtigen Fakten im Bezug zum Oberthema als String zurückgibst.
	Dabei kann es sich beispielsweise um Definitionen von Fachbegriffen, Einführung in Konzepte oder Beispiele handeln. 
	Das Format des Rückgabestrings ist: Fakt1///Fakt2///Fakt3.
	Sei präzise und verzichte auf alles, was nicht zu diesen Faketen gehört.
	Es folgt der Eingabestring: 
	"""
prompt_all = """Du erhälst die Inhalte einer Vorlseung an einer Universität als Eingabestring.
	Du sollst die Vorlesung zusammenfassen, indem du alle akademisch und thematisch wichtigen Fakten im Bezug zum Oberthema als String zurückgibst.
	Das Format des Rückgabestrings ist: Fakt1///Fakt2///Fakt3.
	Sei präzise und verzichte auf alles, was nicht zu diesen Faketen gehört.
	Es folgt der Eingabestring: 
	"""

facts=""
for elem in data:
	facts += generate_text(prompt_all+elem)
	time.sleep(10)
print(facts[:200])

Prompt tokens: 6169 | Thoughts tokens: 1968 | Output tokens: 323 | Total tokens: 8460
Prompt tokens: 5517 | Thoughts tokens: 2343 | Output tokens: 388 | Total tokens: 8248
Prompt tokens: 1654 | Thoughts tokens: 1693 | Output tokens: 113 | Total tokens: 3460
Strategische Spiele sind 2-Spieler, feindliche, sequentielle Spiele mit perfekter Information.///Spielbäume repräsentieren die Zustände und möglichen Züge in strategischen Spielen.///Das Minimax-Prinz


#### Create Flashcards from Information

In [14]:
f_prompt = """Eine Karteikarte besteht aus genau einer Frage und einer Antwort.
	Du sollst Karteikarten erstellen aus den Informationen im Eingabestring.
	Basierend auf diesem String sollst du alle sinnvollen Fragen stellen, die man zu den akademischen Inhalten stellen kann, und diese beantworten.
	Jede Zeile deiner Ausgabe muss folgendem Format entsprechen:  Frage1;Antwort1///Frage2;Antwort2///Frage3;Antwort3.
	Verwende keine Semikolons in der Frage oder in der Antwort.
	Beginne Fragen und Antworten direkt, nicht mit dem Wort Frage oder Antwort.
	Halte dich strikt an das Format Frage1;Antwort1///Frage2;Antwort2///Frage3;Antwort3
	Es folgt der Eingabestring: 
"""
f_prompt_all = """Du sollst Karteikarten erstellen aus den Informationen im Eingabestring.
	Der Eingabestring besteht aus Fakten im Format: Fakt1///Fakt2///Fakt3.
	Für jeden Fakt im Eingabestring sollst du mindestens eine sinnvolle Frage stellen und diese beantworten.
	Jede Zeile deiner Ausgabe muss folgendem Format entsprechen:  Frage1;Antwort1///Frage2;Antwort2///Frage3;Antwort3.
	Verwende keine Semikolons in der Frage oder in der Antwort.
	Beginne Fragen und Antworten direkt, nicht mit dem Wort Frage oder Antwort.
	Halte dich strikt an das Format Frage1;Antwort1///Frage2;Antwort2///Frage3;Antwort3
	Es folgt der Eingabestring: 
"""

res = ""
for elem in [facts]:
	res += generate_text(f_prompt_all+elem)
	time.sleep(10)
print(res[:200])

Prompt tokens: 1005 | Thoughts tokens: 3661 | Output tokens: 1024 | Total tokens: 5690
Was sind die Merkmale strategischer Spiele?;Strategische Spiele sind 2-Spieler feindliche sequentielle Spiele mit perfekter Information.
Was repräsentieren Spielbäume?;Spielbäume repräsentieren Zustän


In [16]:
res_list = res.replace("\n", "///").replace(" /// ", "///").split("///")
f = open(LLM_OUTPUT+"/"+str(filename)[:-4]+"_anki.txt", "w")
failed = 0
for elem in res_list:
	try:
		question, answer = elem.split(";")
		if question[-1:] != "?":
			question += "?"
		if answer[-1:] != ".":
			answer += "."
		f.write(question+";"+answer+"\n")
	except:
		failed += 1
f.close()
if failed > 0:
	print("failed to write "+str(failed)+" elements.")
else:
	print("Successfully created "+str(len(res_list))+" flash cards.")

Successfully created 42 flash cards.
